In [47]:
import cv2 as cv
import numpy as np
print(cv.__version__)

4.5.5


In [48]:
def show_wait_destroy(winname, img):
    cv.namedWindow(winname, cv.WINDOW_NORMAL)
    cv.imshow(winname, img)
    cv.resizeWindow(winname, 400, 300)
    cv.moveWindow(winname, 380, 200)
    cv.waitKey(0)
    cv.destroyWindow(winname)

In [73]:
image = cv.imread('sudoku_3.jpg')
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
show_wait_destroy('img', gray)

In [74]:
gray = cv.bitwise_not(gray)
bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, \
                                cv.THRESH_BINARY, 15, -2)
show_wait_destroy("binary", bw)

In [75]:

horizontal = np.copy(bw)
vertical = np.copy(bw)
cols = horizontal.shape[1]
horizontal_size = cols // 10
horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
# Apply morphology operations
horizontal = cv.erode(horizontal, horizontalStructure)
horizontal = cv.dilate(horizontal, horizontalStructure)
# Show extracted horizontal lines
show_wait_destroy("horizontal", horizontal)

In [76]:
rows = vertical.shape[0]
verticalsize = rows // 10
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv.getStructuringElement(cv.MORPH_RECT, (1, verticalsize))
# Apply morphology operations
vertical = cv.erode(vertical, verticalStructure)
vertical = cv.dilate(vertical, verticalStructure)
# Show extracted vertical lines
show_wait_destroy("vertical", vertical)

In [77]:
bitwiseOr = cv2.bitwise_or(horizontal,vertical)
show_wait_destroy("bitwiseOr", bitwiseOr)


In [78]:
squares = cv.bitwise_not(bitwiseOr)
show_wait_destroy("squares", squares)

In [57]:
merge = cv.bitwise_and(squares, bw)
show_wait_destroy("merge", merge)

In [65]:
import math
img = cv.imread("sudoku.png")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
show_wait_destroy("gray", gray)
edges = cv.Canny(gray, 100,100)
show_wait_destroy("edges", edges)
lines = cv.HoughLinesP(edges, 1, math.pi/2, 2, None, 10, 1)
print(len(lines))
for line in lines[0]:
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    cv.line(img, pt1, pt2, (0,0,255), 3)
show_wait_destroy("merge", img)

456


In [99]:
import cv2
image = cv2.imread('sudoku_2.png')
result = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
show_wait_destroy("thresh", thresh)
cols = horizontal.shape[1]
horizontal_size = cols // 5
# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
show_wait_destroy("detect_horizontal", detect_horizontal)
cnts_h = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_h = cnts_h[0] if len(cnts_h) == 2 else cnts_h[1]
# print(cnts_h)
for c in cnts_h:
    cv2.drawContours(result, [c], -1, (36,255,12), 2)
    
show_wait_destroy("result-1", result)
rows = vertical.shape[0]
verticalsize = rows // 5
# Detect vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,verticalsize))
detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
show_wait_destroy("detect_horizontal", detect_vertical)
cnts_v = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_v = cnts_v[0] if len(cnts_v) == 2 else cnts_v[1]
for c in cnts_v:
    cv2.drawContours(result, [c], -1, (36,255,12), 2)
    
show_wait_destroy("result-2", result)

In [69]:
show_wait_destroy("result", result)

In [111]:
print(cnts_h[6])
print(cnts_v[6])

[[[  5   2]]

 [[  5   3]]

 [[224   3]]

 [[224   2]]]
[[[  3   2]]

 [[  3 221]]

 [[  3 113]]

 [[  4 112]]

 [[  4   2]]]


In [118]:
out = cv2.rectangle(result, (37,2),(3,38), (0, 0, 255), 2)
show_wait_destroy("result", out)

In [122]:
for c in cnts_h[0]:
    print(c[0][0])

5
5
224
224


In [156]:
flat_list =[]
for list in cnts_v:
    for sublist in list:
        flat_list.append(sublist[0][0])
print(flat_list)

[221, 221, 222, 222, 186, 186, 149, 149, 113, 113, 76, 76, 39, 39, 3, 3, 3, 4, 4]


In [149]:
f1 = [sublist[0][1] for lists in cnts_h for sublist in lists]
h_max = max(f1)
h_min = min(f1)


In [153]:
print(h_max)
print(h_min)
h_splits = (h_max+h_min)//len(cnts_h)
print(h_splits)

221
2
31


In [152]:
show_wait_destroy("result", image)